In [1]:
import pull_bondret_treasury
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path
from settings import config

OUTPUT_DIR = Path(config("OUTPUT_DIR"))
DATA_DIR = Path(config("DATA_DIR"))

In [10]:
df = pull_bondret_treasury.load_bondret_treasury_file(data_dir=DATA_DIR)
df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")
df['tr_return'] = df['tr_return'] / 100
df['tr_ytm_match'] = (1 + df['tr_ytm_match'] / 100) ** (1 / 12) - 1
df['spread'] = df['tr_return'] - df['tr_ytm_match']

df = df.dropna(subset=["spread"])
df = df.sort_values(by="DATE")
df["decile"] = df.groupby("DATE")["spread"].transform(lambda x: pd.qcut(x, 10, labels=False))

In [11]:
df

,DATE,CUSIP,tr_return,tr_ytm_match,tau,spread,decile
6,2002-08-31,000336AE7,0.018357,0.002832,5.756164,0.015525,4
1528498,2002-08-31,617446GM5,0.027543,0.003367,8.627397,0.024176,7
1528452,2002-08-31,617446GL7,0.012321,0.002283,3.624658,0.010039,3
1528449,2002-08-31,617446GJ2,0.011726,0.003053,5.583562,0.008673,2
1528205,2002-08-31,617446DX4,0.025117,0.003212,7.794521,0.021904,6
...,...,...,...,...,...,...,...
168305,2023-12-31,045487AB1,0.007004,0.003937,1.043836,0.003067,0
1171047,2023-12-31,42809HAC1,0.071388,0.003280,16.052055,0.068108,7
1788105,2023-12-31,718549AH1,0.029202,0.003155,5.961644,0.026047,4
1787495,2023-12-31,718547AT9,0.019096,0.003232,3.920548,0.015864,3


In [13]:
display(df)

,DATE,CUSIP,tr_return,tr_ytm_match,tau,spread,decile
6,2002-08-31,000336AE7,0.018357,0.002832,5.756164,0.015525,4
1528498,2002-08-31,617446GM5,0.027543,0.003367,8.627397,0.024176,7
1528452,2002-08-31,617446GL7,0.012321,0.002283,3.624658,0.010039,3
1528449,2002-08-31,617446GJ2,0.011726,0.003053,5.583562,0.008673,2
1528205,2002-08-31,617446DX4,0.025117,0.003212,7.794521,0.021904,6
...,...,...,...,...,...,...,...
168305,2023-12-31,045487AB1,0.007004,0.003937,1.043836,0.003067,0
1171047,2023-12-31,42809HAC1,0.071388,0.003280,16.052055,0.068108,7
1788105,2023-12-31,718549AH1,0.029202,0.003155,5.961644,0.026047,4
1787495,2023-12-31,718547AT9,0.019096,0.003232,3.920548,0.015864,3
